In [34]:
import woslite_client
import json

In [35]:
from __future__ import print_function
import time
import woslite_client
from woslite_client.rest import ApiException
from pprint import pprint


# Configure API key authorization: key
configuration = woslite_client.Configuration()
configuration.api_key['X-ApiKey'] = '7263295125113a07f088770334f9f70748c1ac2a'

In [120]:
#create an instance of the API class
integration_api_instance = woslite_client.IntegrationApi(woslite_client.ApiClient(configuration))
search_api_instance = woslite_client.SearchApi(woslite_client.ApiClient(configuration))
database_id = 'WOS'  # str | Database to search. Must be a valid database ID, one of the following: BCI/BIOABS/BIOSIS/CCC/DCI/DIIDW/MEDLINE/WOK/WOS/ZOOREC. WOK represents all databases.
usr_query = 'ALL=(energy) AND PUBL=(Elsevier)'  # str | User query for requesting data, ex: TS=(cadmium). The query parser will return errors for invalid queries.
count = 100 # int | Number of records returned in the request
first_record = 1  # int | Specific record, if any within the result set to return. Cannot be less than 1 and greater than 100000.
publish_time_span = '2022-01-01+2022-12-31'
lang = 'en'  # str | Language of search. This element can take only one value: en for English. If no language is specified, English is passed by default. (optional)
sort_field = 'PY+D'  # str | Order by field(s). Field name and order by clause separated by '+', use A for ASC and D for DESC, ex: PY+D. Multiple values are separated by comma. (optional)

## only get doi

In [37]:
def get_doi(response):
    dl = []
    for i in response.data:
        doi = i.other.identifier_doi
        x_doi = i.other.identifier_xref_doi
        if doi != None:
            dl.append(doi[0])
        else:
            if x_doi != None:
                dl.append(x_doi[0])       
    return dl

In [100]:
total_doi = []

In [121]:
from tqdm import tqdm


try:
    api_response = search_api_instance.root_get(database_id, usr_query, count, first_record, lang=lang,
                                                         sort_field=sort_field, publish_time_span=publish_time_span)
    query_id = api_response.query_result.query_id
    print(api_response.query_result.records_found)
    doi_100 = get_doi(api_response)
    total_doi.extend(doi_100)
    for i in tqdm(range(101, api_response.query_result.records_found, 100)):
        api_response = search_api_instance.query_query_id_get(query_id, 100, i)
        doi_post = get_doi(api_response)
        total_doi.extend(doi_post)

except ApiException as e:
    print("Exception when calling SearchApi->root_get: {}".format(e))

  0%|          | 0/764 [00:00<?, ?it/s]

76452


100%|██████████| 764/764 [04:52<00:00,  2.61it/s]


In [122]:
print(len(total_doi))

387934


In [123]:
# print(api_response.query_result.records_found)
print(len(total_doi))
tmp = list(set(total_doi))
print(len(total_doi))

json_str = json.dumps(total_doi, indent=4)
with open('Energy_Elsevier_1922.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)

387934
387934


## get doi with year

In [4]:
def get_doi_year(response):
    dl = {}
    for i in response.data:
        doi = i.other.identifier_doi
        x_doi = i.other.identifier_xref_doi
        year = i.source.published_biblio_year
        if year != None:
            if doi != None:
                dl[doi[0]]=year[0]
            else:
                if x_doi != None:
                    dl[x_doi[0]]=year[0]           
    return dl

In [30]:
total_doi_year = {}

In [31]:
from tqdm import tqdm


try:
    api_response = search_api_instance.root_get(database_id, usr_query, count, first_record, lang=lang,
                                                         sort_field=sort_field, publish_time_span=publish_time_span)
    query_id = api_response.query_result.query_id
    print(api_response.query_result.records_found)
    doi_100 = get_doi(api_response)
    total_doi_year.update(doi_100)
    for i in tqdm(range(101, api_response.query_result.records_found, 100)):
        api_response = search_api_instance.query_query_id_get(query_id, 100, i)
        doi_post = get_doi(api_response)
        total_doi_year.update(doi_post)

except ApiException as e:
    print("Exception when calling SearchApi->root_get: {}".format(e))

  0%|          | 0/973 [00:00<?, ?it/s]

97303


100%|██████████| 973/973 [06:05<00:00,  2.66it/s]


In [32]:
print(len(total_doi_year))

77363


In [18]:
json_str = json.dumps(total_doi_year, indent=4)
with open('total_doi_year.json', 'w', encoding='utf-8') as json_file:
    json_file.write(json_str)